# Forward

## Generate se3 R6 from SE3 Matrix by Sophus Library

Rotation matrix
```
SE3 Matrix
[ -0.1746031, -0.0317460,  0.9841270;    1
   0.3492065, -0.9365079,  0.0317460;    2
   0.9206349,  0.3492065,  0.1746031 ]   3
      0            0          0          1 
```
(SE3 Matrix ==> se3 R6)
```python
from sophus import *
from sympy import *
q = Quaternion(0.1259882, Matrix([0.6299408, 0.1259882, 0.7559289]))
xi_SE3 = Se3(So3(q), Matrix([1, 2, 3]))
xi_se3 = SE3.log()
```
xi se3 R6
```
se3 R6
Matrix([[3.48075536975048], [2.34718513399708], [0.874839482491055], [1.83448188157597], [0.366896492801180], [2.20137808316218]])
```
Combine Tg pose with xi
```
Tg = 1, 2, 3, 0.6324555, 0.3162278, 0.6324555, 0.3162278
T_combine = xi_SE3_M * Tg_SE3_M
```
T_combine
```
R^4x4
 [-0.615872887393401, 0.768254002808266, -0.174603012694796, 3.71428578273958], 
 [-0.768253903444096, -0.536507791771389, 0.349206445794498, 0.571428888078200], 
 [0.174603364513569, 0.349206351769077, 0.920634830550402, 5.14285714642664], 
 [0, 0, 0, 1]])
 
 R7
 3.71428578273958 0.5714288880782 5.14285714642664 0.4383   -0.0000   -0.1992   -0.8765
```

In [2]:
import torch 

import numpy as np
from SE3Comp import *

batchSize = 2
Tg = torch.zeros(batchSize, 7, 1)
xi_vec = torch.zeros(batchSize, 6, 1)

# Tg[0, 0] = 1
# Tg[0, 1] = 2
# Tg[0, 2] = 3
# Tg[0, 3] = 0.6324555
# Tg[0, 4] = 0.3162278
# Tg[0, 5] = 0.6324555
# Tg[0, 6] = 0.3162278

Tg[0, 0] = 0
Tg[0, 1] = 0
Tg[0, 2] = 0
Tg[0, 3] = 0
Tg[0, 4] = 0
Tg[0, 5] = 0
Tg[0, 6] = 0
# xi_vec[0,0] = 3.48075536975048
# xi_vec[0,1] = 2.34718513399708
# xi_vec[0,2] = 0.874839482491055
# xi_vec[0,3] = 1.83448188157597
# xi_vec[0,4] = 0.366896492801180
# xi_vec[0,5] = 2.20137808316218

xi_vec[0,0] = 0
xi_vec[0,1] = 0
xi_vec[0,2] = 0
xi_vec[0,3] = 0
xi_vec[0,4] = 0
xi_vec[0,5] = 0

model = SE3Comp()


ans = model.forward(Tg, xi_vec)
print(ans)
#3.71428578273958 0.5714288880782 5.14285714642664 0.4383   -0.0000   -0.1992   -0.8765

R
tensor([[[ 1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  1.]],

        [[ 1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  1.]]])
V
tensor([[[ 1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  1.]],

        [[ 1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  1.]]])
Txi
tensor([[ 0.,  0.,  0.],
        [ 0.,  0.,  0.]])
tensor([[[ 0.],
         [ 0.],
         [ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.]],

        [[ 0.],
         [ 0.],
         [ 0.],
         [ 1.],
         [ 0.],
         [ 0.],
         [ 0.]]])


# Backward

In [34]:
import torch.optim as optim

class testNet(nn.Module):
    def __init__(self):
        super(testNet, self).__init__()
        self.se3comp = SE3Comp()
        self.linearTg = nn.Linear(7, 7)
        self.linearXI = nn.Linear(6, 6)
        self.linear3 = nn.Linear(7, 7)
    def forward(self, Tg, xi_vec):
        batchSize = 2
        
        outTg = self.linearTg(Tg.view(batchSize, 1, -1))
        outXI = self.linearXI(xi_vec.view(batchSize,1, -1))
        outC = self.se3comp(outTg.view(batchSize,7,-1) , outXI.view(batchSize, 6, -1))
        outC_squeezed = torch.squeeze(outC)
        
        
        outL = self.linear3(outC_squeezed.view(batchSize, 1, -1))
        
        return outL.view(batchSize, 7, -1)
        
model = testNet()

optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
criterion  = nn.MSELoss()

print('Before optimization\n')
for param in model.parameters():
    print(param.data)
print('==============================\n')    

Tans = torch.zeros(batchSize, 7, 1)
Tg[0, 0] = 1
Tg[0, 1] = 2
Tg[0, 2] = 0
Tg[0, 3] = 0
Tg[0, 4] = 0.3162278
Tg[0, 5] = 0.6324555
Tg[0, 6] = 0.3162278

EPOCH = 50
for k in range(EPOCH):
    optimizer.zero_grad()
    output = model(Tg, xi_vec)
    #print('output size: ', output.size())

    loss = criterion(output, Tans)
    print(loss)
    loss.register_hook(print)
    loss.backward
    optimizer.step()

print('After optimization\n')
for param in model.parameters():
    print(param.data)

Before optimization

tensor([[-0.1696,  0.2081, -0.2086, -0.2917, -0.2273, -0.3017, -0.0339],
        [ 0.3304,  0.3171,  0.2417, -0.2813,  0.0459,  0.1605,  0.1279],
        [ 0.1945, -0.2777, -0.0612, -0.0163,  0.2093, -0.1172, -0.1393],
        [ 0.2085, -0.1922,  0.1244,  0.2897,  0.1418,  0.3760, -0.1734],
        [ 0.3445,  0.0452, -0.3564, -0.1274, -0.0022, -0.3309, -0.1582],
        [ 0.1029, -0.1571,  0.1368, -0.3428, -0.1953, -0.3509, -0.0734],
        [-0.0304,  0.0749, -0.2983,  0.3132, -0.0953,  0.2934,  0.0013]])
tensor([-0.1659, -0.2764,  0.2008,  0.0346,  0.1733, -0.1328,  0.3282])
tensor([[ 0.3257, -0.0104, -0.4051,  0.1441,  0.3979, -0.2569],
        [ 0.2838,  0.2279, -0.1175, -0.3361,  0.2237,  0.2689],
        [ 0.0681,  0.2872,  0.3464, -0.0017, -0.1268, -0.2446],
        [-0.0105, -0.1128,  0.2570,  0.2927,  0.1576, -0.2875],
        [-0.1254,  0.1184, -0.2634,  0.1554,  0.0376, -0.0598],
        [ 0.1583,  0.2524, -0.3112, -0.1510,  0.2820,  0.3935]])
tensor([-0

In [36]:
import torch 

import numpy as np
from SE3Comp import *

batchSize = 2
q = torch.zeros(batchSize, 4, 1)
q[:,0] = 0
q[:,1] = 0
q[:,2] = 0
q[:,3] = 0


def q_to_Matrix(q):
    qw = q[:, 0]
    qx = q[:, 1]
    qy = q[:, 2]
    qz = q[:, 3]
    M = torch.zeros(q.size()[0], 3, 3)

    M[:, 0, 0] = torch.squeeze( 1 - 2*torch.mul(qy,qy) - 2*torch.mul(qz,qz) )
    M[:, 1, 0] = torch.squeeze( 2*torch.mul(qx,qy) + 2*torch.mul(qz,qw) )
    M[:, 2, 0] = torch.squeeze( 2*torch.mul(qx,qz) - 2*torch.mul(qy,qw) )

    M[:, 0, 1] = torch.squeeze( 2*torch.mul(qx,qy) - 2*torch.mul(qz,qw) )
    M[:, 1, 1] = torch.squeeze( 1 - 2*torch.mul(qx,qx) - 2*torch.mul(qz,qz) )
    M[:, 2, 1] = torch.squeeze( 2*torch.mul(qy,qz) + 2*torch.mul(qx,qw) )

    M[:, 0, 2] = torch.squeeze( 2*torch.mul(qx,qz) + 2*torch.mul(qy,qw) )
    M[:, 1, 2] = torch.squeeze( 2*torch.mul(qy,qz) - 2*torch.mul(qx,qw) )
    M[:, 2, 2] = torch.squeeze( 1 - 2*torch.mul(qx,qx) - 2*torch.mul(qy,qy) )

    return M

ans = q_to_Matrix(q)
print(ans)

tensor([[[ 1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  1.]],

        [[ 1.,  0.,  0.],
         [ 0.,  1.,  0.],
         [ 0.,  0.,  1.]]])


In [93]:
import numpy as np
np.set_printoptions(precision=15)
torch.set_printoptions(precision=15)
theta = torch.from_numpy(np.array([1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]))

co = (1.0 - torch.cos(theta)) / (theta*theta)
print('by formula')
print(co)

co_l = torch.full((theta.shape[0],), 1.0 / 2.0, dtype=torch.float64)
co_l += torch.pow(theta, (4*1)) / 720.0# 6!
co_l += theta**(4*2) / 3628800.0# 10!
co_l -= theta**(2) / 24.0
co_l -= theta**(2 + 4) / 40320.0

print('by Taylor')
print(co_l)





by formula
tensor([0.499583472197418, 0.499995833347366, 0.499999958325503, 0.499999996961265, 0.500000041370185, 0.500044450291171, 0.499600361081320,
        0.000000000000000, 0.000000000000000, 0.000000000000000], dtype=torch.float64)
by Taylor
tensor([0.499583472197423, 0.499995833347222, 0.499999958333335, 0.499999999583333, 0.499999999995833, 0.499999999999958, 0.500000000000000,
        0.500000000000000, 0.500000000000000, 0.500000000000000], dtype=torch.float64)


In [96]:
si = (theta - torch.sin(theta)) / (theta**3)
print(si)

si_l = torch.full((theta.shape[0],), 1.0 / 6.0, dtype=torch.float64)
si_l += theta**(4*1) / 5040.0
si_l += theta**(4*2) / 39916800.0
si_l -= theta**(2) / 120.0
si_l -= theta**(2 + 4) / 362880.0
print(si_l)

tensor([0.166583353171851, 0.166665833335744, 0.166666658339004, 0.166666661483190, 0.166667284899440, 0.166653732372284, 0.172053567411030,
        0.000000000000000, 0.000000000000000, 0.000000000000000], dtype=torch.float64)
tensor([0.166583353171848, 0.166665833335317, 0.166666658333334, 0.166666666583333, 0.166666666665833, 0.166666666666658, 0.166666666666667,
        0.166666666666667, 0.166666666666667, 0.166666666666667], dtype=torch.float64)


In [67]:
co_l_pos = torch.full((theta.shape[0],), 1.0 / 2.0)

In [75]:
np.math.factorial(9) 

362880

In [81]:
theta = torch.from_numpy(np.array([1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9, 1e-10]))
one = torch.sin(theta) / theta
print(one)

tensor([ 0.9983,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
         1.0000,  1.0000,  1.0000], dtype=torch.float64)
